**Prithibi Paul | Group 6 | Question 6**  
Grouping Sets  
GROUPING SETS Subclause  
CUBE Subclause  
ROLLUP Subclause  
GROUPING and GROUPING\_ID Function

**Question 6:**

In [ ]:
-- Question: 6
-- Write a query against the dbo.Orders table that returns the 
-- total quantities for each:
-- employee, customer, and order year
-- employee and order year
-- customer and order year
-- Include a result column in the output that uniquely identifies 
-- the grouping set with which the current row is associated
-- Tables involved: TSQLV4 database, dbo.Orders table

-- Desired output:
groupingset empid       custid orderyear   sumqty
----------- ----------- ------ ----------- -----------
0           2           A      2014        12
0           3           A      2014        10
4           NULL        A      2014        22
0           2           A      2015        40
4           NULL        A      2015        40
0           3           A      2016        10
4           NULL        A      2016        10
0           1           B      2014        20
4           NULL        B      2014        20
0           2           B      2015        12
4           NULL        B      2015        12
0           2           B      2016        15
4           NULL        B      2016        15
0           3           C      2014        22
4           NULL        C      2014        22
0           1           C      2015        14
4           NULL        C      2015        14
0           1           C      2016        20
4           NULL        C      2016        20
0           3           D      2016        30
4           NULL        D      2016        30
2           1           NULL   2014        20
2           2           NULL   2014        12
2           3           NULL   2014        32
2           1           NULL   2015        14
2           2           NULL   2015        52
2           1           NULL   2016        20
2           2           NULL   2016        15
2           3           NULL   2016        40


## Proposition
Write a SQL query that computes, for each order in the `dbo.Orders` table, the difference in quantities between the current order and the customer's previous and next orders. This query will assist in analyzing order quantity trends for each customer.

## Tables Involved
1. `dbo.Orders`: Contains basic information about orders.
2. `dbo.OrderDetails`: Contains details about the orders, including quantities.

## Columns Used
- `custid` (from `dbo.Orders`): The customer ID associated with each order.
- `orderid` (from `dbo.Orders`): The unique identifier for each order.
- `qty` (from `dbo.OrderDetails`): The quantity of items in each order.
- `diffprev` (calculated): The difference in quantity between the current order and the customer's previous order.
- `diffnext` (calculated): The difference in quantity between the current order and the customer's next order.

## Query Logic
- **Join**: The query joins the `dbo.Orders` table with the `dbo.OrderDetails` table on the `orderid` to combine the order information with its corresponding details.
- **Partitioning and Ordering**: The `LAG` window function is used, partitioned by `custid` and ordered by `orderid`, to calculate `diffprev`, the difference between the current order quantity and the previous order's quantity for the same customer. Similarly, the `LEAD` window function is used to calculate `diffnext`, the difference between the current order quantity and the next order's quantity.


In [46]:
USE TSQLV4;
SELECT
    o.EmpId AS EmployeeId,
    o.CustId AS CustomerId,
    YEAR(o.OrderDate) AS OrderYear,
    SUM(od.Qty) AS TotalQuantity,
    CASE 
        WHEN GROUPING(o.EmpId) = 1 AND GROUPING(o.CustId) = 1 THEN 'Year Only'
        WHEN GROUPING(o.EmpId) = 1 THEN 'Customer and Year'
        WHEN GROUPING(o.CustId) = 1 THEN 'Employee and Year'
        ELSE 'Employee, Customer and Year'
    END AS GroupingSet
FROM
    dbo.Orders AS o
JOIN 
    dbo.OrderDetails AS od ON o.OrderId = od.OrderId
GROUP BY 
    GROUPING SETS (
        (o.EmpId, o.CustId, YEAR(o.OrderDate)),
        (o.EmpId, YEAR(o.OrderDate)),
        (o.CustId, YEAR(o.OrderDate)),
        (YEAR(o.OrderDate))
    )
ORDER BY
    GroupingSet, o.EmpId, o.CustId, OrderYear;

-- Use Northwind db

USE Northwinds2022TSQLV7;
SELECT
    o.EmployeeId,
    o.CustomerId,
    YEAR(o.OrderDate) AS OrderYear,
    SUM(od.Quantity) AS TotalQuantity,
    -- Calculate the difference between the current total order quantity
    -- and the previous total order quantity for the same employee and customer
    SUM(od.Quantity) - LAG(SUM(od.Quantity)) OVER(PARTITION BY o.EmployeeId, o.CustomerId ORDER BY YEAR(o.OrderDate)) AS DiffPrevYear,
    -- Calculate the difference between the current total order quantity
    -- and the next total order quantity for the same employee and customer
    LEAD(SUM(od.Quantity)) OVER(PARTITION BY o.EmployeeId, o.CustomerId ORDER BY YEAR(o.OrderDate)) - SUM(od.Quantity) AS DiffNextYear
FROM 
    Sales.[Order] AS o
JOIN 
    Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY 
    o.EmployeeId, 
    o.CustomerId, 
    YEAR(o.OrderDate)
ORDER BY 
    o.EmployeeId, 
    o.CustomerId, 
    OrderYear;

(917 rows affected)

(653 rows affected)

Total execution time: 00:00:00.073

EmployeeId,CustomerId,OrderYear,TotalQuantity,GroupingSet
NULL,1,2015,79,Customer and Year
NULL,1,2016,95,Customer and Year
NULL,2,2014,6,Customer and Year
NULL,2,2015,28,Customer and Year
NULL,2,2016,29,Customer and Year
NULL,3,2014,24,Customer and Year
NULL,3,2015,295,Customer and Year
NULL,3,2016,40,Customer and Year
NULL,4,2014,105,Customer and Year
NULL,4,2015,371,Customer and Year


EmployeeId,CustomerId,OrderYear,TotalQuantity,DiffPrevYear,DiffNextYear
1,1,2016,35,NULL,NULL
1,3,2015,38,NULL,NULL
1,4,2015,164,NULL,NULL
1,5,2015,235,NULL,NULL
1,9,2014,72,NULL,-27
1,9,2015,45,-27,-9
1,9,2016,36,-9,NULL
1,10,2016,77,NULL,NULL
1,11,2016,34,NULL,NULL
1,14,2015,83,NULL,NULL


**Part 2 of Hw - Prithibi Paul  
Chapter 7 - Beyond the Fundamentals of Querying**

```
---------------------------------------------------------------------
-- Grouping Sets
---------------------------------------------------------------------
```

In [ ]:
-- Unifying result sets of four queries
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY empid, custid

UNION ALL

SELECT empid, NULL, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY empid

UNION ALL

SELECT NULL, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY custid

UNION ALL

SELECT NULL, NULL, SUM(qty) AS sumqty
FROM dbo.Orders;

# Proposition

Create a SQL query that aggregates the quantities from the `dbo.Orders` table in four different ways and unifies the results into a single dataset. The query should calculate the total quantity for each combination of employee and customer, each employee individually, each customer individually, and for all orders combined.

## Tables Involved

- `dbo.Orders`: Contains order details including employee ID, customer ID, and quantity.

## Columns Used

- `empid` (from `dbo.Orders`): The employee ID associated with each order.
- `custid` (from `dbo.Orders`): The customer ID associated with each order.
- `qty` (from `dbo.Orders`): The quantity of items in each order.
- `sumqty` (calculated): The total quantity of orders, aggregated in different ways.

## Query Logic

- The query aggregates the total quantity (`sumqty`) from the `dbo.Orders` table for different groupings: by employee and customer, by employee only, by customer only, and for all orders.
- These aggregations are unified into a single result set using the `UNION ALL` operator, ensuring that all combinations are represented.


In [47]:
USE Northwinds2022TSQLV7
SELECT o.EmployeeId, o.CustomerId, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY o.EmployeeId, o.CustomerId

UNION ALL

SELECT o.EmployeeId, NULL, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY o.EmployeeId

UNION ALL

SELECT NULL, o.CustomerId, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY o.CustomerId

UNION ALL

SELECT NULL, NULL, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId;

(563 rows affected)

Total execution time: 00:00:00.069

EmployeeId,CustomerId,sumqty
3,50,12
2,34,84
9,8,40
4,68,152
2,11,50
3,4,103
3,27,9
4,91,27
6,79,99
7,66,20


```
---------------------------------------------------------------------
-- GROUPING SETS Subclause
---------------------------------------------------------------------
```

In [ ]:
-- Using the GROUPING SETS subclause
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY
  GROUPING SETS
  (
    (empid, custid),
    (empid),
    (custid),
    ()
  );

# Proposition

Construct a SQL query to aggregate order quantities in the `dbo.Orders` table using the `GROUPING SETS` subclause. This query will calculate the total quantity of orders for different groupings: each combination of employee and customer, each employee alone, each customer alone, and a grand total for all orders.

## Tables Involved

- `dbo.Orders`: Contains order details, including employee ID, customer ID, and quantity.

## Columns Used

- `empid` (from `dbo.Orders`): The employee ID associated with each order.
- `custid` (from `dbo.Orders`): The customer ID associated with each order.
- `qty` (from `dbo.Orders`): The quantity of items in each order.
- `sumqty` (calculated): The total aggregated quantity of orders for each grouping set.

## Query Logic

- The query utilizes the `GROUPING SETS` subclause to aggregate the total quantity (`sumqty`) from the `dbo.Orders` table for various combinations: employee with customer, employee alone, customer alone, and a total aggregation for all orders.

In [48]:
USE Northwinds2022TSQLV7
SELECT o.EmployeeId, o.CustomerId, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY
  GROUPING SETS
  (
    (o.EmployeeId, o.CustomerId),
    (o.EmployeeId),
    (o.CustomerId),
    ()
  );

(563 rows affected)

Total execution time: 00:00:00.048

EmployeeId,CustomerId,sumqty
1,1,35
3,1,60
4,1,41
6,1,38
NULL,1,174
3,2,28
4,2,29
7,2,6
NULL,2,63
1,3,38


```
---------------------------------------------------------------------
-- CUBE Subclause
---------------------------------------------------------------------
```

In [ ]:
-- Using the CUBE subclause
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);

# Proposition

Develop a SQL query that utilizes the `CUBE` subclause to perform multidimensional analysis on the `dbo.Orders` table. This query will aggregate the total quantity of orders, allowing analysis of the data across multiple dimensions – by each employee, by each customer, and by each combination of employee and customer, along with the overall total.

## Tables Involved

- `dbo.Orders`: Contains order details such as employee ID, customer ID, and order quantities.

## Columns Used

- `empid` (from `dbo.Orders`): Employee ID associated with each order.
- `custid` (from `dbo.Orders`): Customer ID associated with each order.
- `qty` (from `dbo.Orders`): Quantity of items in each order.
- `sumqty` (calculated): Total aggregated quantity for each grouping level provided by the `CUBE`.

## Query Logic

- The query uses the `CUBE` subclause in the `GROUP BY` statement to perform aggregation of the quantity (`qty`) across multiple dimensions: individually by `empid` and `custid`, combined, and the overall total.


In [49]:
USE Northwinds2022TSQLV7
SELECT o.EmployeeId, o.CustomerId, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY CUBE(o.EmployeeId, o.CustomerId);


(563 rows affected)

Total execution time: 00:00:00.046

EmployeeId,CustomerId,sumqty
1,1,35
3,1,60
4,1,41
6,1,38
NULL,1,174
3,2,28
4,2,29
7,2,6
NULL,2,63
1,3,38


```
---------------------------------------------------------------------
-- ROLLUP Subclause
---------------------------------------------------------------------
```

In [ ]:
-- Using the ROLLUP subclause
SELECT 
  YEAR(orderdate) AS orderyear,
  MONTH(orderdate) AS ordermonth,
  DAY(orderdate) AS orderday,
  SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY ROLLUP(YEAR(orderdate), MONTH(orderdate), DAY(orderdate));

# Proposition

Formulate a SQL query employing the `ROLLUP` subclause to analyze the `dbo.Orders` table hierarchically. This query aims to aggregate the total quantity of orders, broken down by year, month, and day, allowing for a detailed yet hierarchical view of order quantities over time.

## Tables Involved

- `dbo.Orders`: This table includes details such as the order date and quantities of orders.

## Columns Used

- `orderdate` (from `dbo.Orders`): The date when the order was placed.
- `qty` (from `dbo.Orders`): Quantity of items in each order.
- `sumqty` (calculated): The total aggregated quantity of orders at each level of the date hierarchy.

## Query Logic

- The query uses the `ROLLUP` subclause in the `GROUP BY` statement to enable hierarchical aggregation of `qty` by year, month, and day of the `orderdate`.


In [50]:
USE Northwinds2022TSQLV7
SELECT 
  YEAR(o.OrderDate) AS OrderYear,
  MONTH(o.OrderDate) AS OrderMonth,
  DAY(o.OrderDate) AS OrderDay,
  SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY ROLLUP(YEAR(o.OrderDate), MONTH(o.OrderDate), DAY(o.OrderDate));


(507 rows affected)

Total execution time: 00:00:00.038

OrderYear,OrderMonth,OrderDay,sumqty
2014,7,4,27
2014,7,5,49
2014,7,8,101
2014,7,9,105
2014,7,10,102
2014,7,11,57
2014,7,12,110
2014,7,15,27
2014,7,16,46
2014,7,17,121


```
---------------------------------------------------------------------
-- GROUPING and GROUPING_ID Function
---------------------------------------------------------------------
```

In [ ]:
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);

SELECT
  GROUPING(empid) AS grpemp,
  GROUPING(custid) AS grpcust,
  empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);

SELECT
  GROUPING_ID(empid, custid) AS groupingset,
  empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);

# Proposition

Craft a SQL query utilizing the `GROUPING` and `GROUPING_ID` functions along with the `CUBE` subclause on the `dbo.Orders` table. This query will aggregate order quantities and identify the level of aggregation using these functions, providing insights into the data aggregation across multiple dimensions – both individually and combined for employee and customer.

## Tables Involved

- `dbo.Orders`: Contains order details, including employee ID, customer ID, and order quantities.

## Columns Used

- `empid` (from `dbo.Orders`): Employee ID associated with each order.
- `custid` (from `dbo.Orders`): Customer ID associated with each order.
- `qty` (from `dbo.Orders`): Quantity of items in each order.
- `sumqty` (calculated): Total aggregated quantity for each group.
- `grpemp` and `grpcust` (calculated): Indicators (0 or 1) showing whether the row is a subtotal or grand total.

## Query Logic

- The queries use the `CUBE` subclause in the `GROUP BY` statement for multidimensional aggregation of `qty`.
- The `GROUPING` function is used to identify whether a row is a subtotal or a grand total for `empid` and `custid`.
- The `GROUPING_ID` function is used to provide a unique identifier for each grouping set created by `CUBE`.


In [51]:
-- Using CUBE for aggregation:

USE Northwinds2022TSQLV7
SELECT o.EmployeeId, o.CustomerId, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY CUBE(o.EmployeeId, o.CustomerId);


-- Using GROUPING function with CUBE:
SELECT
  GROUPING(o.EmployeeId) AS GrpEmp,
  GROUPING(o.CustomerId) AS GrpCust,
  o.EmployeeId, o.CustomerId, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY CUBE(o.EmployeeId, o.CustomerId);


-- Using GROUPING_ID function with CUBE:
SELECT
  GROUPING_ID(o.EmployeeId, o.CustomerId) AS GroupingSet,
  o.EmployeeId, o.CustomerId, SUM(od.Quantity) AS sumqty
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od ON o.OrderId = od.OrderId
GROUP BY CUBE(o.EmployeeId, o.CustomerId);


(563 rows affected)

(563 rows affected)

(563 rows affected)

Total execution time: 00:00:00.106

EmployeeId,CustomerId,sumqty
1,1,35
3,1,60
4,1,41
6,1,38
NULL,1,174
3,2,28
4,2,29
7,2,6
NULL,2,63
1,3,38


GrpEmp,GrpCust,EmployeeId,CustomerId,sumqty
0,0,1,1,35
0,0,3,1,60
0,0,4,1,41
0,0,6,1,38
1,0,NULL,1,174
0,0,3,2,28
0,0,4,2,29
0,0,7,2,6
1,0,NULL,2,63
0,0,1,3,38


GroupingSet,EmployeeId,CustomerId,sumqty
0,1,1,35
0,3,1,60
0,4,1,41
0,6,1,38
2,NULL,1,174
0,3,2,28
0,4,2,29
0,7,2,6
2,NULL,2,63
0,1,3,38


<span style="color: rgba(0, 0, 0, 0.87); font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255);">Written&nbsp;</span> <span style="font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255); color: rgb(0, 0, 255);">in</span> <span style="color: rgba(0, 0, 0, 0.87); font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255);">&nbsp;collaboration&nbsp;</span> <span style="font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255); color: rgb(0, 0, 255);">with</span> <span style="color: rgba(0, 0, 0, 0.87); font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255);">&nbsp;ChatGPT&nbsp;</span> <span style="font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255); color: rgb(0, 0, 255);">from</span> <span style="color: rgba(0, 0, 0, 0.87); font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255);">&nbsp;OpenAI&nbsp;</span> <span style="font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255); color: rgb(0, 0, 255);">to</span> <span style="color: rgba(0, 0, 0, 0.87); font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255);">&nbsp;improve understanding&nbsp;</span> <span style="font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255); color: rgb(0, 0, 255);">and</span> <span style="color: rgba(0, 0, 0, 0.87); font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255);">&nbsp;assist&nbsp;</span> <span style="font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255); color: rgb(0, 0, 255);">with</span> <span style="color: rgba(0, 0, 0, 0.87); font-family: system-ui, -apple-system, blinkmacsystemfont, &quot;Segoe UI&quot;, helvetica, arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; background-color: rgb(255, 255, 255);">&nbsp;the explanation of the query.</span>